In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from typing import List, Tuple
from preprocessing import (
    clsf_boz_cd, clsf_rrnr_dvcd, clsf_dmfr_dvcd, clsf_crd_grd, clsf_pdgr_cd,
    clsf_cntr_catg_cd, get_disc_factor_all, get_loss_adj_rate_all
)

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_일반')
BASE_YYMM = '202012'

In [73]:
# 데이터 불러오기
## 보험료부채 익스포져
일반_원수_미경과보험료 = pd.read_excel(FILE_PATH / f'일반_원수_미경과보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_미경과보험료 = pd.read_excel(FILE_PATH / f'일반_출재_미경과보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_예정보험료 = pd.read_excel(FILE_PATH / f'일반_원수_예정보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_예정보험료 = pd.read_excel(FILE_PATH / f'일반_출재_예정보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str})

## 준비금부채 익스포져
일반_원수_개별추산액 = pd.read_excel(FILE_PATH / f'일반_원수_개별추산액_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_개별추산액 = pd.read_excel(FILE_PATH / f'일반_출재_개별추산액_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})

## 기초정보, 가정 등
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
일반_보험금진전추이 = pd.read_excel(FILE_PATH / f'일반_보험금진전추이_{BASE_YYMM}.xlsx', dtype={'PDGR_CD': str, 'AY': str})
일반_최종손해율 = pd.read_excel(FILE_PATH / f'일반_최종손해율_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'PDGR_CD': str}) \
        .rename(columns= {'RRNR_DAT_DVCD': 'RRNR_DVCD'})
일반_사업비율 = pd.read_excel(FILE_PATH / f'일반_사업비율_{BASE_YYMM}.xlsx', dtype={'RRNR_DVCD': str, 'DMFR_DVCD': str}) \
        .assign(BOZ_CD = lambda x: x['BOZ_CD'].str.replace('A100', '#'))
일반_IBNR = pd.read_excel(FILE_PATH / f'일반_IBNR_{BASE_YYMM}.xlsx', dtype={'RRNR_DVCD': str, 'IBNR_KEY': str}) \
        .drop('CLG_YM', axis=1) \
        .assign(RRNR_DVCD = lambda x: x['RRNR_DVCD'].str.pad(width=2))

## 공통
국가그룹 = pd.read_excel(FILE_PATH / '국가그룹.xlsx', dtype={'CNTR_CATG_CD': str})
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / f'재보험자_국내신용등급_{BASE_YYMM}.xlsx', dtype={'재보험사코드': str}) \
        .rename(columns = {'재보험사코드': 'T02_RN_RINSC_CD', '국내신용등급': 'CRD_GRD'})
선도부도율 = pd.read_excel(FILE_PATH / f'선도부도율_{BASE_YYMM}.xlsx')
할인율 = pd.read_excel(FILE_PATH / f'할인율_{BASE_YYMM}.xlsx')

In [80]:
# 데이터 전처리
## 미경과보험료
일반_원수_미경과보험료_가공 = 일반_원수_미경과보험료.copy()
일반_원수_미경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_미경과보험료_가공, '원수')
일반_원수_미경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_미경과보험료_가공)
일반_원수_미경과보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_원수_미경과보험료_가공, 국가그룹)
일반_원수_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_미경과보험료_가공, 일반_상품정보)
일반_원수_미경과보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_미경과보험료_가공, 일반_상품정보)
일반_원수_미경과보험료_가공['CRD_GRD'] = '#'
일반_원수_미경과보험료_가공.rename(columns = {'LTPD_URND_PRM': 'URND_PRM'}, inplace=True)
일반_원수_미경과보험료_가공.drop(['RRNR_DAT_DVCD', 'RRNR_CTC_BZ_DVCD', 'NTNL_CTRY_CD', 'ARC_INPL_CD'], axis=1, inplace=True)

일반_출재_미경과보험료_가공 = 일반_출재_미경과보험료.copy()
일반_출재_미경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_미경과보험료_가공, '출재')
일반_출재_미경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_미경과보험료_가공)
일반_출재_미경과보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_출재_미경과보험료_가공, 국가그룹)
일반_출재_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_미경과보험료_가공, 일반_상품정보)
일반_출재_미경과보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_미경과보험료_가공, 일반_상품정보)
일반_출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_미경과보험료_가공, 재보험자_국내신용등급)
일반_출재_미경과보험료_가공.rename(columns = {'T02_LTPD_RN_URND_PRM': 'URND_PRM'}, inplace=True)
일반_출재_미경과보험료_가공.drop(['RRNR_DAT_DVCD', 'RRNR_CTC_BZ_DVCD', 'ARC_INPL_CD', 'NTNL_CTRY_CD', 'T02_RN_RINSC_CD'], axis=1, inplace=True)

일반_미경과보험료_가공 = pd.concat([일반_원수_미경과보험료_가공, 일반_출재_미경과보험료_가공], axis=0).reset_index(drop=True)

## 예정보험료
일반_원수_예정보험료_가공 = 일반_원수_예정보험료.copy()
일반_원수_예정보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_예정보험료_가공, '원수')
일반_원수_예정보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_예정보험료_가공)
일반_원수_예정보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_원수_예정보험료_가공, 국가그룹)
일반_원수_예정보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_예정보험료_가공, 일반_상품정보)
일반_원수_예정보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_예정보험료_가공, 일반_상품정보)
일반_원수_예정보험료_가공['CRD_GRD'] = '#'
일반_원수_예정보험료_가공.rename(columns = {'WNCR_CNV_EPCT_PRM': 'EPCT_PRM'}, inplace=True)
일반_원수_예정보험료_가공.drop(['RRNR_DAT_DVCD', 'RRNR_CTC_BZ_DVCD', 'ARC_INPL_CD', 'NTNL_CTRY_CD'], axis=1, inplace=True)

일반_출재_예정보험료_가공 = 일반_출재_예정보험료.copy()
일반_출재_예정보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_예정보험료_가공, '출재')
일반_출재_예정보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_예정보험료_가공)
일반_출재_예정보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_출재_예정보험료_가공, 국가그룹)
일반_출재_예정보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_예정보험료_가공, 일반_상품정보)
일반_출재_예정보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_예정보험료_가공, 일반_상품정보)
일반_출재_예정보험료_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_예정보험료_가공, 재보험자_국내신용등급)
일반_출재_예정보험료_가공.rename(columns = {'WNCR_CNV_RN_PRM': 'EPCT_PRM'}, inplace=True)
일반_출재_예정보험료_가공.drop(['ARC_INPL_CD', 'RRNR_CTC_BZ_DVCD', 'RRNR_DAT_DVCD', 'NTNL_CTRY_CD', 'T02_RN_RINSC_CD'], axis=1, inplace=True)

일반_예정보험료_가공 = pd.concat([일반_원수_예정보험료_가공, 일반_출재_예정보험료_가공], axis=0).reset_index(drop=True)

## 개별추산액
일반_원수_개별추산액_가공 = 일반_원수_개별추산액.copy()
일반_원수_개별추산액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_개별추산액_가공, '원수')
일반_원수_개별추산액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_개별추산액_가공)
일반_원수_개별추산액_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_원수_개별추산액_가공, 국가그룹)
일반_원수_개별추산액_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_개별추산액_가공, 일반_상품정보)
일반_원수_개별추산액_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_개별추산액_가공, 일반_상품정보)
일반_원수_개별추산액_가공['CRD_GRD'] = '#'
일반_원수_개별추산액_가공.rename(columns = {'LTPD_OST_AMT': 'OST_AMT'}, inplace=True)
일반_원수_개별추산액_가공.drop(['RRNR_DAT_DVCD', 'RRNR_DMFR_DVCD', 'NTNL_CTRY_CD', 'ARC_INPL_CD'], axis=1, inplace=True)

일반_출재_개별추산액_가공 = 일반_출재_개별추산액.copy()
일반_출재_개별추산액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_개별추산액_가공, '출재')
일반_출재_개별추산액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_개별추산액_가공)
일반_출재_개별추산액_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_개별추산액_가공, 일반_상품정보)
일반_출재_개별추산액_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_개별추산액_가공, 일반_상품정보)
일반_출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_개별추산액_가공, 재보험자_국내신용등급)
일반_출재_개별추산액_가공.rename(columns = {'RN_LTPD_OST_AMT': 'OST_AMT'}, inplace=True)
일반_출재_개별추산액_가공.drop(['RRNR_DAT_DVCD', 'RRNR_DMFR_DVCD', 'NTNL_CTRY_CD', 'ARC_INPL_CD', 'T02_RN_RINSC_CD'], axis=1, inplace=True)

일반_개별추산액_가공 = pd.concat([일반_원수_개별추산액_가공, 일반_출재_개별추산액_가공], axis=0).reset_index(drop=True)

## 직전3년보험금손조비
일반_원수_직전3년보험금손조비_가공 = 일반_원수_직전3년보험금손조비.copy()
일반_원수_직전3년보험금손조비_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년보험금손조비_가공, '원수')
일반_원수_직전3년보험금손조비_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년보험금손조비_가공)
일반_원수_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년보험금손조비_가공, 일반_상품정보)
일반_원수_직전3년보험금손조비_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_직전3년보험금손조비_가공, 일반_상품정보)
일반_원수_직전3년보험금손조비_가공['CRD_GRD'] = '#'
일반_원수_직전3년보험금손조비_가공.rename(columns = {'ACCD_IVMT': 'DAG_IVMT', 'OGL_PYN_BNF': 'PYN_BNF'}, inplace=True)
일반_원수_직전3년보험금손조비_가공.drop(['RRNR_DAT_DVCD', 'RRNR_DMFR_DVCD', 'ARC_INPL_CD'], axis=1, inplace=True)

일반_출재_직전3년보험금손조비_가공 = 일반_출재_직전3년보험금손조비.copy()
일반_출재_직전3년보험금손조비_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_직전3년보험금손조비_가공, '출재')
일반_출재_직전3년보험금손조비_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년보험금손조비_가공)
일반_출재_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년보험금손조비_가공, 일반_상품정보)
일반_출재_직전3년보험금손조비_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_직전3년보험금손조비_가공, 일반_상품정보)
일반_출재_직전3년보험금손조비_가공['CRD_GRD'] = '#'
일반_출재_직전3년보험금손조비_가공.rename(columns = {'WNCR_CNV_RN_DAG_IVMT': 'DAG_IVMT', 'WNCR_CNV_PYN_BNF': 'PYN_BNF'}, inplace=True)
일반_출재_직전3년보험금손조비_가공.drop(['RRNR_DAT_DVCD', 'RRNR_DMFR_DVCD', 'ARC_INPL_CD'], axis=1, inplace=True)

일반_직전3년보험금손조비_가공 = pd.concat([일반_원수_직전3년보험금손조비_가공, 일반_출재_직전3년보험금손조비_가공], axis=0).reset_index(drop=True)

In [87]:
# 가정산출
## 최종손해율
### 8개 상품군(화재, 기술, 근재, 책임, 종합, 상해, 기타, 해상) 가정 사용
일반_최종손해율 = 일반_최종손해율.loc[lambda x: x['PDGR_CD'].isin(['23', '24', '25', '26', '27', '28', '29', '31'])].reset_index(drop=True)

## 손해조사비율
### 1. 원수+수재, 출재로 그룹핑
### 2. PDGR_CD 단위로 그룹핑
### 3. 해외의 경우 PDGR_CD 구분없이 산출
일반_손해조사비율 = 일반_직전3년보험금손조비_가공.copy()
일반_손해조사비율.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
일반_손해조사비율.loc[lambda x: x['DMFR_DVCD']=='02', 'PDGR_CD'] = '#'
일반_손해조사비율 = 일반_손해조사비율.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD'])[['DAG_IVMT', 'PYN_BNF']].sum() \
        .eval('LAE_RATIO = DAG_IVMT/PYN_BNF') \
        .reset_index() \
        .drop(['DAG_IVMT', 'PYN_BNF'], axis=1)
일반_손해조사비율_국내 = 일반_손해조사비율.loc[lambda x: x['DMFR_DVCD']=='01']
일반_손해조사비율_해외 = pd.concat([일반_손해조사비율.loc[lambda x: x['DMFR_DVCD']=='02'].copy().assign(PDGR_CD = pdgr_cd) for pdgr_cd in ['23', '24', '25', '26', '27', '28', '29', '31']])
일반_손해조사비율 = pd.concat([일반_손해조사비율_국내, 일반_손해조사비율_해외], axis=0)
일반_원수_손해조사비율 = 일반_손해조사비율.loc[lambda x: x['RRNR_DVCD'] == '04'].copy().assign(RRNR_DVCD = '01')
일반_수재_손해조사비율 = 일반_손해조사비율.loc[lambda x: x['RRNR_DVCD'] == '04'].copy().assign(RRNR_DVCD = '02')
일반_출재_손해조사비율 = 일반_손해조사비율.query('RRNR_DVCD == "03"').copy()
일반_손해조사비율 = pd.concat([일반_원수_손해조사비율, 일반_수재_손해조사비율, 일반_출재_손해조사비율]).reset_index(drop=True)

## 할인요소
일반_할인요소 = get_disc_factor_all(일반_보험금진전추이, 할인율.query('KICS_SCEN_NO == 1'))

## 재보험자산 손실조정율
손실조정율 = get_loss_adj_rate_all(일반_보험금진전추이, 할인율.query('KICS_SCEN_NO == 1'), 선도부도율)

## IBNR
### DMFR_DVCD, PDGR_DVCD, RRNR_DVCD(원수+수재는 묶어서) 단위로 배분
일반_IBNR_가공 = 일반_IBNR.copy()
일반_IBNR_가공 = 일반_IBNR_가공.assign(
    PDGR_DVCD = lambda x: x['IBNR_KEY'].str.split('_').map(lambda x: x[1] if len(x)==2 else '#'),
    DMFR_DVCD = lambda x: x['IBNR_KEY'].str.split('_').map(lambda x: f'{int(x[0]):02}')
)
일반_IBNR_가공.rename(columns = {'KRW_OS_AMT_E': 'IBNR'}, inplace=True)
일반_IBNR_가공.drop('IBNR_KEY', axis=1, inplace=True)
일반_IBNR_가공 = 일반_IBNR_가공.query('RRNR_DVCD != "02"') \
    .assign(RRNR_DVCD = lambda x: np.where(x['RRNR_DVCD']=='01', '04', x['RRNR_DVCD'])) 

In [96]:
일반_원수_미경과보험료_가공

,INER_CHN_DVCD,URND_PRM,RRNR_DVCD,DMFR_DVCD,CNTR_CATG_CD,BOZ_CD,PDGR_CD,CRD_GRD
0,nan,0,01,01,01,A010,29,#
1,nan,170557730,01,01,01,A004,31,#
2,11,3616664,02,02,06,A002,24,#
3,11,43297663,02,02,02,A003,27,#
4,11,17439194,02,02,06,A004,31,#
...,...,...,...,...,...,...,...,...
400,nan,2903709,01,01,01,A002,24,#
401,nan,4031664,01,01,01,A007,28,#
402,11,0,02,02,07,A006,26,#
403,11,0,02,01,01,A009,29,#


In [92]:
일반_원수_개별추산액_가공

,OST_AMT,RRNR_DVCD,DMFR_DVCD,CNTR_CATG_CD,BOZ_CD,PDGR_CD,CRD_GRD
0,101941927456,01,01,01,A003,27,#
1,5332823905,01,01,01,A005,25,#
2,1601032713,01,01,01,A006,26,#
3,1189500000,01,01,01,A010,29,#
4,712404197,01,01,01,A006,26,#
...,...,...,...,...,...,...,...
339,20683594,02,02,07,A004,31,#
340,64984,02,02,05,A004,31,#
341,1299137,02,02,07,A003,27,#
342,134411,02,02,07,A007,28,#


In [91]:
일반_IBNR_가공

,RRNR_DVCD,IBNR,PDGR_DVCD,DMFR_DVCD
0,04,0,23,01
1,04,"1,935,683,285",24,01
2,04,"15,692,007,593",25,01
3,04,"18,884,093,455",26,01
4,04,"2,656,264,186",27,01
5,04,"49,548,884,618",28,01
6,04,"18,267,608,620",29,01
7,04,"6,595,105,160",31,01
8,04,"141,078,869,842",#,02
10,03,0,23,01


In [10]:
# 데이터 집계
KEYS = ['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD', 'CNTR_CATG_CD', 'CRD_GRD', 'BOZ_CD']
일반_미경과보험료_집계 = 일반_미경과보험료_가공.groupby(KEYS)['URND_PRM'].sum().reset_index()
일반_예정보험료_집계 = 일반_예정보험료_가공.groupby(KEYS)['EPCT_PRM'].sum().reset_index()
일반_보험료부채 = 일반_미경과보험료_집계 \
    .merge(일반_예정보험료_집계, on=KEYS, how='outer') \
    .assign(URND_PRM = lambda x: x['URND_PRM'].fillna(0).astype(float)) \
    .assign(EPCT_PRM = lambda x: x['EPCT_PRM'].fillna(0).astype(float)) \
    .merge(일반_최종손해율, on=['RRNR_DVCD', 'PDGR_CD'], how='left') \
    .merge(일반_손해조사비율, on=['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD'], how='left') \
    .merge(일반_할인요소[['PDGR_CD', 'DISC_FAC_PRM']], on=['PDGR_CD'], how='left') \
    .merge(손실조정율[['RRNR_DVCD', 'PDGR_CD', 'CRD_GRD', 'LOSS_ADJ_RATE_PRM']], on=['RRNR_DVCD', 'PDGR_CD', 'CRD_GRD'], how='left') \
    .assign(LOSS_ADJ_RATE_PRM = lambda x: x['LOSS_ADJ_RATE_PRM'].fillna(0).astype(float))
일반_보험료부채 = 일반_보험료부채.eval('MAINT_EXP_RATIO = 0.03') # 삭제예정
일반_보험료부채 = 일반_보험료부채.eval('MAINT_EXP = (URND_PRM+EPCT_PRM)*MAINT_EXP_RATIO') # 삭제예정
일반_보험료부채 = 일반_보험료부채 \
        .eval('FNAL_LOSS = (URND_PRM+EPCT_PRM)*FNAL_LSRT') \
        .eval('LAE = FNAL_LOSS*LAE_RATIO') \
        .eval('LIAB_PRM = -EPCT_PRM+FNAL_LOSS+LAE+MAINT_EXP') \
        .eval('PV_LIAB_PRM = LIAB_PRM*DISC_FAC_PRM') \
        .eval('LOSS_ADJ_PRM = LIAB_PRM*LOSS_ADJ_RATE_PRM')
일반_보험료부채.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
일반_보험료부채.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_보험료부채_집계 = 일반_보험료부채.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])[['PV_LIAB_PRM', 'LOSS_ADJ_PRM']].sum().reset_index()

In [11]:
# 데이터 내보내기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/일반_현행추정부채_{now}.xlsx', 'xlsxwriter') as writer:
    일반_보험료부채_집계.to_excel(writer, '일반_보험료부채', index=False)